In [1]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer('multi-qa-distilbert-cos-v1')

c:\Users\a.ramirez.lopez\Documents\Study\Repositories\llm-zoomcamp-2024\.conda\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
c:\Users\a.ramirez.lopez\Documents\Study\Repositories\llm-zoomcamp-2024\.conda\lib\site-packages\threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\a.ramirez.lopez\Documents\Study\Repositories\llm-zoomcamp-2024\.conda\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\a.ramirez.lopez\.cache\huggingface\hub\models--sentence-transformers--multi-qa-distilbert-cos-v1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.52k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Q1. Getting the embeddings model

In [2]:
user_question = "I just discovered the course. Can I still join it?"

In [18]:
v = embedding_model.encode([user_question])[0]
v[0]

0.07822266

# Prepare documents

In [5]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [13]:
documents = list(filter(lambda x: x['course'] == "machine-learning-zoomcamp", documents))

In [14]:
len(documents)

375

# Q2. Creating the embeddings

In [15]:
documents[0]

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'section': 'General course-related questions',
 'question': 'How do I sign up?',
 'course': 'machine-learning-zoomcamp',
 'id': '0227b872'}

In [16]:
embeddings = []

for document in documents:
    qa_text = f'{document["question"]} {document["text"]}'
    embedding = embedding_model.encode([qa_text])[0]
    embeddings.append(embedding)

In [17]:
import numpy as np

X = np.array(embeddings)
X.shape

(375, 768)

# Q3. Search

In [21]:
scores = X.dot(v)
max(scores)

0.6506573

# Vector search

In [24]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(v, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

# Q4. Hit-rate for our search engine

In [34]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [39]:
from tqdm import tqdm


def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)


def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)


def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [40]:
def numpy_cosine_search(q):
    question = q['question']
    v_q = embedding_model.encode(question)
    results = search_engine.search(v_q, num_results=5)
    return results


evaluate(ground_truth, numpy_cosine_search)

100%|██████████| 1830/1830 [01:28<00:00, 20.60it/s]


{'hit_rate': 0.9398907103825137, 'mrr': 0.8516484517304189}

# Q5. Indexing with Elasticsearch

In [49]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 
es_client.info()

ObjectApiResponse({'name': '25c4ab37dbf3', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'fAREqLjQStSZe7qN2w_V1A', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [50]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "text_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"},
        }
    }
}

In [51]:
index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [53]:
for doc, embedding in zip(documents, embeddings):
    doc["text_vector"] = embedding
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

In [54]:
query = {
    "field": "text_vector",
    "query_vector": v,
    "k": 5,
    "num_candidates": 10000, 
}

In [60]:
res = es_client.search(index=index_name, knn=query, source=["text", "section", "question", "course"])
results = res["hits"]["hits"]

max_score = max([result["_score"] for result in results])

[result["_id"] for result in results if result["_score"] == max_score]

['WFk67pABI-q9NkzRkUsy']

# Q6. Hit-rate for Elasticsearch

In [72]:
def es_cosine_search(q):
    question = q['question']
    v_q = embedding_model.encode(question)

    query = {
        "field": "text_vector",
        "query_vector": v_q,
        "k": 5,
        "num_candidates": 10000, 
    }
    res = es_client.search(index=index_name, knn=query, source=["id", "text", "section", "question", "course"])
    results = res["hits"]["hits"]

    results = [result["_source"] for result in results]
    return results

In [73]:
evaluate(ground_truth, es_cosine_search)

100%|██████████| 1830/1830 [05:44<00:00,  5.30it/s]  


{'hit_rate': 0.9349726775956284, 'mrr': 0.8478415300546447}